In [2]:
import json
import os
import pandas as pd

import requests

# Here we store our API read key in a string variable that we can reference later.
baseurl = 'https://api.purpleair.com/'
my_api_read_key = os.environ.get('PURPLE_AIR_API_KEY_READ')
if my_api_read_key is None:
    raise ValueError("PURPLE_AIR_API_KEY_READ environment variable not set")
read_headers = {'X-API-Key':my_api_read_key ,
           'Content-Type': 'application/json'}
my_api_write_key = os.environ.get('PURPLE_AIR_API_KEY_WRITE')
if my_api_write_key is None:
    raise ValueError("PURPLE_AIR_API_KEY_WRITE environment variable not set")
write_headers = {'X-API-Key':my_api_write_key ,
           'Content-Type': 'application/json'}

def getSensorData(sensor_index):

    # my_url is assigned the URL we are going to send our request to.
    my_url = 'https://api.purpleair.com/v1/sensors/' + str(sensor_index)

    # my_headers is assigned the context of our request we want to make. In this case
    # we will pass through our API read key using the variable created above.
    my_headers = {'X-API-Key':my_api_read_key}

    # my_params is assigned a list of fields of data we are requesting. Excluding the
    # fields parameter will collect all available fields. In this example, we ask for the
    # temperature and the PM2.5 ATM average.
    my_params = {'fields':'temperature,pm2.5_atm'}

    # This line creates and sends the request and then assigns its response to the
    # variable, r.
    r = requests.get(my_url, headers=my_headers, params=my_params)

    # We then return the response we received.
    return r

https://api.purpleair.com/#api-sensors-get-sensors-data

In [ ]:
def getSensorData(sensor_index):

    # my_url is assigned the URL we are going to send our request to.
    my_url = 'https://api.purpleair.com/v1/sensors/' + str(sensor_index)

    # my_headers is assigned the context of our request we want to make. In this case
    # we will pass through our API read key using the variable created above.
    my_headers = {'X-API-Key':my_api_read_key}

    # my_params is assigned a list of fields of data we are requesting. Excluding the
    # fields parameter will collect all available fields. In this example, we ask for the
    # temperature and the PM2.5 ATM average.
    my_params = {'fields':'temperature,pm2.5_atm'}

    # This line creates and sends the request and then assigns its response to the
    # variable, r.
    r = requests.get(my_url, headers=my_headers, params=my_params)

    # We then return the response we received.
    return r

Create a group
http://api.purpleair.com/v1/groups HTTP/1.1
api_key: my-api-write-key
Content-Type: application/json

{
"name": "My Group"
}

In [4]:
create_group_url = f'{baseurl}v1/groups'
group_params={ "name": "Resilient_SouthRegion" }
r = requests.post(create_group_url, data=json.dumps(group_params), headers=write_headers, )
if r.status_code == 200 or r.status_code == 201:
    groups = r.text
    j = json.loads(groups)
    group_id = j["group_id"]
    print ( group_id)
else: 
    print (r.status_code)

2644


{
  "api_version" : "V1.0.14-0.0.58",
  "time_stamp" : 1744994865,
  "group_id": 2634
}


In [9]:
get_sensors_url = f'{baseurl}v1/sensors'
fields=["sensor_index",
        "last_modified",
        "name",
        "latitude",
        "longitude",
        "location_type"
        ]
fields = ",".join(fields)

sensors_params={ "nwlat":32.7,"nwlng":-117.2,"selat":32.5,"selng":-117.0, 'fields':fields }

r = requests.get(get_sensors_url, headers=read_headers, params=sensors_params)
if r.status_code == 200 or r.status_code == 201:
    sensors = r.json()
    print ( sensors)
    df = pd.DataFrame(sensors["data"], columns=sensors["fields"])
    df_outside=df[df['location_type']==0]
    print(df_outside)
else:
    print(r.status_code)
    print(r.text)

{'api_version': 'V1.0.14-0.0.58', 'time_stamp': 1745271307, 'data_time_stamp': 1745271286, 'max_age': 604800, 'firmware_default_version': '7.02', 'fields': ['sensor_index', 'last_modified', 'name', 'location_type', 'latitude', 'longitude'], 'location_types': ['outside', 'inside'], 'data': [[21583, 1666717264, 'N 4 | Pase Médico II | AYUNTAMIENTO DE TIJUANA', 0, 32.537693, -117.02728], [21593, 1709330852, '10 | Subdelegación Francisco Villa | AYUNTAMIENTO DE TIJUANA', 0, 32.51655, -117.05975], [22393, 1569080387, 'NBC7 (Chula Vista)', 0, 32.573006, -117.035126], [36473, 1652030399, 'Logan Heights Post Office', 0, 32.696865, -117.134186], [75275, 1724979960, 'The Airborne Institute #1', 0, 32.69682, -117.18461], [82281, 1712348177, 'Mathewson Park, Coronado', 0, 32.68735, -117.17259], [120333, 1669345806, 'LOWER SURREY DRIVE', 0, 32.65409, -117.03505], [137488, 1657222431, 'N°1 | Palacio Municipal | AYUNTAMIENTO DE TIJUANA', 0, 32.531906, -117.01893], [161215, 1674509883, 'EMK2', 0, 32.6

In [11]:
add_members_url = f'{baseurl}v1/groups/{group_id}/members'
for member in df["sensor_index"]:
    add_members_params={ "group_id":group_id , 'sensor_index': member}
    r = requests.post(add_members_url, headers=write_headers, data=json.dumps(add_members_params))
    if r.status_code == 200 or r.status_code == 201:
        print(r.text)
    else:
        print(r.status_code)
        print(r.text)
        

{
  "api_version" : "V1.0.14-0.0.58",
  "time_stamp" : 1744996795,
  "data_time_stamp" : 1744996766,
  "group_id" : 2638,
  "member_id" : 235550,
  "sensor" : {
    "sensor_index" : 21583,
    "last_modified" : 1666717264,
    "date_created" : 1544573286,
    "last_seen" : 1744996695,
    "private" : 0,
    "is_owner" : 0,
    "name" : "N 4 | Pase Médico II | AYUNTAMIENTO DE TIJUANA",
    "icon" : 0,
    "location_type" : 0,
    "model" : "PA-II",
    "hardware" : "2.0+PMSX003-B+PMSX003-A",
    "led_brightness" : 35,
    "firmware_version" : "7.02",
    "rssi" : -72,
    "uptime" : 16887,
    "pa_latency" : 375,
    "memory" : 16528,
    "position_rating" : 5,
    "latitude" : 32.537693,
    "longitude" : -117.02728,
    "altitude" : 92,
    "channel_state" : 3,
    "channel_flags" : 0,
    "channel_flags_manual" : 0,
    "channel_flags_auto" : 0,
    "confidence" : 100,
    "confidence_auto" : 100,
    "confidence_manual" : 100,
    "analog_input" : 0.03,
    "pm1.0" : 2.8,
    "pm1.0

In [14]:

get_members_data_url = f'{baseurl}v1/groups/{group_id}/members'
add_members_url = f'{baseurl}v1/groups/{group_id}/members'

fields=["sensor_index","pm2.5","pm2.5_60minute","pm10.0", "voc", "humidity", "temperature",
        "last_modified",
              "name",
              "latitude",
              "longitude",
              "location_type"
              ]
fields = ",".join(fields)
members_params={ "group_id":group_id,"fields":fields}
r = requests.get(get_members_data_url, headers=read_headers, params=members_params)
if r.status_code == 200 or r.status_code == 201:
    sensors=r.json()
    df = pd.DataFrame(sensors["data"], columns=sensors["fields"])
    
else:
    print(r.status_code)
    print(r.text)

In [18]:
import aqicalc as aqi
df['AQI']= df.apply(lambda row: aqi.to_aqi( [ ( aqi.POLLUTANT_PM25,row.get('pm2.5')) ,
                                              (aqi.POLLUTANT_PM10, row.get('pm10.0'))]
                                            ), axis=1)
df

,sensor_index,last_modified,name,location_type,latitude,longitude,humidity,temperature,voc,pm2.5,pm10.0,pm2.5_60minute,AQI
0,21583,1666717264,N 4 | Pase Médico II | AYUNTAMIENTO DE TIJUANA,0,32.537693,-117.027280,NaN,NaN,NaN,7.9,11.0,12.0,33
1,21593,1709330852,10 | Subdelegación Francisco Villa | AYUNTAMIE...,0,32.516550,-117.059750,NaN,NaN,NaN,4.6,5.2,6.6,19
2,22393,1569080387,NBC7 (Chula Vista),0,32.573006,-117.035126,37.0,85.0,NaN,0.4,0.4,0.6,2
3,36473,1652030399,Logan Heights Post Office,0,32.696865,-117.134186,36.0,76.0,NaN,4.6,5.3,4.6,19
4,75275,1724979960,The Airborne Institute #1,0,32.696820,-117.184610,49.0,69.0,NaN,6.5,7.8,8.7,27
5,82281,1712348177,"Mathewson Park, Coronado",0,32.687350,-117.172590,40.0,78.0,NaN,6.6,9.5,8.5,27
6,120333,1669345806,LOWER SURREY DRIVE,0,32.654090,-117.035050,25.0,93.0,NaN,2.5,3.2,5.1,10
7,137488,1657222431,N°1 | Palacio Municipal | AYUNTAMIENTO DE TIJUANA,0,32.531906,-117.018930,42.0,75.0,NaN,3328.0,3328.0,226.2,3328.0
8,161215,1674509883,EMK2,0,32.623670,-117.133484,NaN,NaN,NaN,6.6,8.8,8.2,27
9,161219,1740770088,La Sierra,0,32.677673,-117.176170,25.0,94.0,NaN,11.6,15.5,13.5,48


In [13]:
df

,sensor_index,last_modified,name,location_type,latitude,longitude,humidity,temperature,voc,pm2.5,pm2.5_60minute
0,21583,1666717264,N 4 | Pase Médico II | AYUNTAMIENTO DE TIJUANA,0,32.537693,-117.027280,NaN,NaN,NaN,8.5,12.2
1,21593,1709330852,10 | Subdelegación Francisco Villa | AYUNTAMIE...,0,32.516550,-117.059750,NaN,NaN,NaN,4.6,6.8
2,22393,1569080387,NBC7 (Chula Vista),0,32.573006,-117.035126,39.0,84.0,NaN,0.1,0.6
3,36473,1652030399,Logan Heights Post Office,0,32.696865,-117.134186,36.0,76.0,NaN,4.6,4.6
4,75275,1724979960,The Airborne Institute #1,0,32.696820,-117.184610,49.0,69.0,NaN,6.9,8.8
5,82281,1712348177,"Mathewson Park, Coronado",0,32.687350,-117.172590,40.0,78.0,NaN,7.0,8.6
6,120333,1669345806,LOWER SURREY DRIVE,0,32.654090,-117.035050,26.0,92.0,NaN,2.6,5.2
7,137488,1657222431,N°1 | Palacio Municipal | AYUNTAMIENTO DE TIJUANA,0,32.531906,-117.018930,42.0,76.0,NaN,3327.5,226.2
8,161215,1674509883,EMK2,0,32.623670,-117.133484,NaN,NaN,NaN,7.0,8.2
9,161219,1740770088,La Sierra,0,32.677673,-117.176170,24.0,95.0,NaN,9.9,13.6
